pip install tensorflow

In [11]:
import os
import sys
sys.path.append("../../")
# openCV
import cv2
# numpy
import numpy as np
# class util functions
#from utils.imutils import jimshow as show
#from utils.imutils import jimshow_channel as show_channel

# cifar10 dataset
from tensorflow.keras.datasets import cifar10

ModuleNotFoundError: No module named 'utils'

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [10]:
X_list=[]

for image in X_train:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
    normalized = gray_image/255 # normalize
    X_list.append(normalized) # add to list
X_preprocess_array = np.array(X_list) # turn list back into an array
# reshape
X_preprocess_array.reshape(-1, 1024)

type(image)

numpy.ndarray